<a href="https://colab.research.google.com/github/srirambandi/ai/blob/master/examples/mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# when running in colab notebooks, first install library
!pip install import-ai
# upload respective dataset manually from examples directory of the library or download as below
!apt install subversion
!svn checkout https://github.com/srirambandi/ai/trunk/examples/MNIST

In [2]:
import ai
import numpy as np

In [3]:
def load_data(file):
    dict = np.load(file, allow_pickle=True)
    return dict

In [4]:
train_file = 'MNIST/train.npy'
test_file = 'MNIST/test.npy'

In [5]:
ai.manual_seed(2357)

In [6]:
class MLP(ai.Module):
    def __init__(self, ):
        self.fc1 = ai.Linear(784, 200)
        self.drop = ai.Dropout(p=0.75)
        self.fc2 = ai.Linear(200, 10)
        
    def forward(self, x):
        o1 = self.drop(ai.G.relu(self.fc1(x)))
        o2 = ai.G.softmax(self.fc2(o1))
        
        return o2

In [7]:
mlp = MLP()
print(mlp)

MLP(
  fc1: Linear(input_features=784, output_features=200, bias=True)
  fc2: Linear(input_features=200, output_features=10, bias=True)
)


In [8]:
L = ai.Loss(loss_fn='CrossEntropyLoss')
optim = ai.Optimizer(mlp.parameters(), optim_fn='Adam', lr=1e-3)

In [ ]:
train_dict = load_data(train_file)
inputs = train_dict.item()['data']
outputs = train_dict.item()['labels']
del train_dict

In [10]:
it, epoch = 0, 0
m = 128

In [11]:
def evaluate():
    ai.G.grad_mode = False
    file = test_file
    dict = load_data(file)
    inputs = dict.item()['data']
    outputs = dict.item()['labels']
    correct, total = 0, 0
    test_m = m
    for batch in range(int(len(outputs) / m)):
        input = inputs[batch * test_m : (batch + 1) * test_m] / 255
        input =  np.stack([_ for _ in input], axis = -1)
        output = np.array(outputs[batch * test_m : (batch + 1) * test_m])
        scores = mlp.forward(input)
        preds = np.argmax(scores.data, axis=0)
        correct += np.sum(np.equal(output, preds))
        total += test_m
    accuracy = float(correct / total)
    ai.G.grad_mode = True
    return accuracy

In [12]:
while epoch < 15:
    epoch += 1
    it = 0

    for batch in range(int(len(outputs) / m)):

        input = inputs[batch * m : (batch + 1) * m] / 255.
        input =  np.stack([_ for _ in input], axis = -1)
        output = outputs[batch * m : (batch + 1) * m]
        onehot = np.zeros((10, m))
        for _ in range(m):
            onehot[output[_], _] = 1.0

        scores = mlp.forward(input)
        loss = L.loss(scores, onehot)
        loss.backward()

        optim.step()        # update parameters with optimization functions
        optim.zero_grad()   # clearing the backprop list and resetting the gradients to zero

        if it%50 == 0:
            print('epoch: {}, iter: {}, loss: {}'.format(epoch, it, loss.data[0, 0]))
        it += 1

    print('Epoch {} completed. Accuracy: {:.2%} \n'.format(epoch, evaluate()))

using Adam
epoch: 1, iter: 0, loss: 2.296190367528439
epoch: 1, iter: 50, loss: 0.630809503335878
epoch: 1, iter: 100, loss: 0.38746586028567753
epoch: 1, iter: 150, loss: 0.42698727058220026
epoch: 1, iter: 200, loss: 0.2784827476327498
epoch: 1, iter: 250, loss: 0.3307337010967575
epoch: 1, iter: 300, loss: 0.27478059463871163
epoch: 1, iter: 350, loss: 0.374498570044405
epoch: 1, iter: 400, loss: 0.318463067741763
epoch: 1, iter: 450, loss: 0.35128901019913006
Epoch 1 completed. Accuracy: 93.63% 

epoch: 2, iter: 0, loss: 0.16343432446845055
epoch: 2, iter: 50, loss: 0.23444355258953284
epoch: 2, iter: 100, loss: 0.17580449529033787
epoch: 2, iter: 150, loss: 0.21282138977291373
epoch: 2, iter: 200, loss: 0.14509753378842705
epoch: 2, iter: 250, loss: 0.2358382822672339
epoch: 2, iter: 300, loss: 0.16274376514809621
epoch: 2, iter: 350, loss: 0.24967028600289634
epoch: 2, iter: 400, loss: 0.18538694088052993
epoch: 2, iter: 450, loss: 0.21965838176576707
Epoch 2 completed. Accuracy:

In [13]:
mlp.save()

saving model...
Successfully saved model in MLP.npy
